## ForJoe

*Some things from my notebook that would be good to have recorded somewhere.*

*I'll add some pictures as soon as I figure out a good way of generating them*

The goal of the project is to verify properties of a closed loop neural network controller. The problem has multiple facets and subproblems to deal with, but the overall objective is to prove a formal property on a system that has this structure:

                input state -> [control NN + dynamics NN] -> output state -> repeat

Or in other words: 
$$ x_t \longrightarrow C(x_t) = c \longrightarrow D(x_t, c) \longrightarrow x_{t+1}$$

That said, this is just one way to think about it in our particular case. The more general problem is simply to show that $$ f_t(X_0) \subseteq Y $$ for some starting set $X_0$, where $f_t$ denotes $t$ applications of $f$. If this property is true, the controller can be said to be stable for $t$ time steps (assuming $Y$ is some set that can be considered "stable"). 

We also want to demonstrate this with an image-based controller that uses an image of the state. I.e.:

$$ x_t \longrightarrow I(x_t) \longrightarrow C(I(x_t)) = c \longrightarrow D(x_t, c) \longrightarrow x_{t+1}$$

The added challenge there is that the mapping $I$ is not one-to-one with $x$. I.e. nearby states can map to the same image. There is another, much greater challenge, that is to do with velocity in an image-based controller. I'll get to that later.

The particular environment we're working on is the inverted pendulum, since it's simple and is a classic reinforcement learning problem (found in OpenAIGym, etc.). The exact architecture of the network is up in the air. In particular, the control network can be fully-connected or convolutional as well as feed-forward or recurrent (ideally, we want to prove a property on all permutations thereof, but not necessarily the same property). 

The exact architecture of the problem is a bit more nuanced than described above due to how the dynamics are overapproximated. See [Chelsea's paper](https://stanford.app.box.com/file/478687098549) for details. In particular, the output of the network is not $x_{t+1}$ in this scheme, but an overapproximated set $X_{t+1}$. That said, for properties that use only a single timestep, this nuance can be temporarily neglected I think.

## Tools
Just to note, while we can use **NeuralVerification.jl** if we work in julia, that package is probably not best suited for the more complex networks we want to show this on, and does not implement the state-of-the-art algorithms. So it's probably not for the best.

So far, Chelsea has mostly used **Marabou** — a verification tool written in C++ (with a python interface) that is the next generation of Reluplex, and the current state of the art for complete solvers. A complete solver is one which does not (over)approximate and therefore never returns false negatives (never states that a property is violated when in fact it is not). While this is useful, it often slows computation down, so we'll see if we can relax it in some cases and still prove properties. Marabou is based on SAT/SMT solvers, which under the hood ultimately maps to solving linear programs, often with some other logic on top. You can check out a bunch of Chelsea's stuff for how to use Marabou with python, including this straightforward test file: https://github.com/guykatzz/Marabou/blob/master/maraboupy/test/multi_input_test.py (if you can't reach it it's because the marabou repo is private... we'll have to get you access).

There is another state-of-the-art solver called **ERAN** from ETH-Zurich which is reachability-based. It is not complete like Marabou but could afford other advantages for some formulations of the problem (it also seems to have a python interface). https://github.com/eth-sri/eran


## Property in state space

In the pendulum problem, the state consists of angle and angular velocity, $x = (\theta, \dot\theta)$ and the control network may output a torque $\ddot\theta$. A finite time safety property in this space could be something like: 
$$X_0 = \{x \mid \theta \in [a, b],\  \dot\theta \in [c, d])\}$$
$$f_t(X_0) \subseteq Y$$ 
for a known safe set $Y$. Notably, if $Y$ is a subset of $X_0$, the safety property holds for all time. I.e., if the property reads,

$$f_t(X_0) = X_1 \subseteq X_0,$$

then subsequent applications of $f$ on $X_1$ are also guaranteed to return to $X_0$, since $X_1$ itself is a subset of $X_0$.

Likewise, if at any $n>0,\ f_{t+n}(X_0) \subseteq f_{t}(X_0)$ then we have a cycle, and therefore stability. This is identical to the previously described property with $X_0 = f_t(X^*_0)$. This fact has been guiding the majority of my thinking on the topic, and I'll say more about it later.

## Properties in general

Note that in the example above, $\theta$ and $\dot\theta$ take on interval values. I mentioned in our conversation that input sets need to be convex for verification. Technically this is not true in general (it is in NV.jl though). I think in the most general case, Marabou can handle any conjunctions or disjunctions of linear constraints on the input and output sets. I'm not terribly familiar with it still, but from Chelsea's code, it looks like setting upper and lower bounds on variables is the primary interface to creating the verification problem. From earlier discussions, I seem to recall that certain logic may also be used (e.g. if-then), but that could have been manual tweeking Chelsea was working with on top of the Marabou interface; not sure, have to defer to Chelsea on that front. In any case, linear constraints should be safe to use in pretty much every context, and intervals are linear constraints, so we're good.

The verification problem for a SAT based solver is to find a point 
$$x \in X \mid f(x) \in Y^c$$
I.e. a point in $X$ that maps to the complement of the desired set. As I understand Marabou (not that well), the set you encode as the output set is $Y^c$, and it simply tries to solve a feasibility problem on the sets $X$ and $Y^c$ using SMT methods. If the problem has a feasible solution, the safety property is violated.

### $Y^c$ as a $\cap$ of linear constraints (convex)

These are polytopes. When $Y^c$ is a polytope, the property can be verified by solving a single satisfiability problem. If 
$$\exists x \in X \mid f(x) \in Y^c,$$
you're done (and the safety property is violated). This kind of propery maps to an "exclusion zone". I.e. a convex region of the state space you *don't* want to be in. These types of problems can be encoded into a single satisfiability problem.

### $Y^c$ as a $\cup$ of linear constraints (complement of convex)

Complement of a polytope. This maps to an "inclusion zone" safety property. I.e. a convex region of the state space you *do* want to be in. In this case, $Y^c$ is not convex, and $Y$ is. Defining $Y^c$ as a union of linear constraints,

$$ Y^c = \bigcup a^T_i y \leq b_i, $$

the property is violated if there exists an assigment for $x$ such that:

$$a^T_1 f(x) > b_1 \ \ \land\ \  a^T_2 f(x) > b_2 \dots \ \land\ \  a^T_n f(x) > b_n$$

I.e. all of the constraints must be violated. If any one of the constraints holds, then the property holds.

Which actually means you need to solve $n$ of these satisfiability problems, one for each constraint$^*$.  -- Actually, with SMT magic, this maybe isn't true; the logical aspects of SMT are maybe able to deal with this directly.

### $Y^c$ as a combination of both $\cap$ and $\cup$ of linear constraints 

Can be broken up into smaller problems of the forms above. Don't think there's a one-step solution for this kind of problem.

---

Just to note, in reachability-based solvers you generally encode $Y$ not $Y^c$. In both cases though, the goal is to show $\forall x \in X, f(x) \in Y$, thereby proving the property.

---

### Relationships between variables

Before, the variables each took on interval values in the input set; i.e. each one was an independent range. It is also easily possible to encode any linear relationship between the variables. I think SMT lets you do certain nonlinear relations as well, but I'm not sure which or how (worth looking in to).

See this notebook from the CARS NeuralVerification workshop for an explanation of this as it pertains to MNIST classification: https://github.com/sisl/NeuralVerification.jl/blob/master/examples/cars_workshop/5%20image%20classification.ipynb

The gist is as follows: at the heart of SMT/SAT solvers are linear programs, which involve finding an x that satisfies linear constraints of the form
$$ A x \leq b $$

$A$ and $b$ can, for example, be something like

$$A = \begin{bmatrix}
1 & 2 \\
0 & 1
\end{bmatrix}
\\
b = \begin{bmatrix}
0\\
2
\end{bmatrix}
$$

Thus, the constraints would come out to be

$$ \begin{align} x_1  + 2x_2 &\leq 0 \\
x_2 &\leq 2 
\end{align}$$

Any linear relationship between the variables $x$ can be encoded in this way. This is how we define either the safe set $Y$, or the unsafe set $Y^c$. Unions of linear constraints can not be represented in this way (but Marabou can still handle them).

In the MNIST example, the property encoded is that the input image be "near" an image of a $1$ in an $L_\infty$ sense. I.e. that the input set is the set of all images whose maximum pixel deviation from the source image is less than some specified value. The output in that example simply has to lie outside the space of $1$s in classification space. You can refer to the matrix $A$ in that example to understand $L_\infty$ norm properties. $L_1$ norm properties can also be encoded linearly. E.g. the sum of the absolute deviations in each pixel needs to be less than some value.

## Forward invariance example 

Consider the input set that is off by up to 10 degrees in either direction of vertical, i.e. $\Theta = [-10^{\circ}, 10^{\circ}]$. Knowing everything about our system, e.g. the maximum control torque, length and mass of the pendulum, etc., we can hypothesize that there is some set of starting velocities $\dot\Theta$ such that: 

$$X_0 = (\Theta, \dot\Theta),$$

$$\forall x \in X_0, f(x) \subseteq X_0$$

I.e. we can say that for some range of velocities, the set is forward invariant. This may even be true for any starting angular deviation. If we find such a set for which the property holds, then as described earlier, the controller can be considered safe for all time if it starts in that set.

However, the range $\dot\Theta$ that makes this true may be quite small for a particular $\Theta$, and may be smaller than the range of velocities for which the controller is safe if we know something more specific about the starting conditions. I.e. if we could make $\dot\theta$ a function of $\theta$ we could say something more general. Unfortunately, we can't do this unless the relation is linear (maybe with SMT we can?).

One thing we can try is to partition the set $X_0$ and prove two smaller properties that together may be more general. Take a set with positive $\theta$ and negative $\dot\theta$; i.e. 

$$\begin{align}
\Theta^+ &=  0 \leq \theta \leq a \\
\dot\Theta^- &= -b \leq \dot\theta \leq 0\\
X_0^+ &= (\Theta^+, \dot\Theta^-) \\
X_0^- &= (\Theta^-, \dot\Theta^+)
\end{align}$$

We can ask, for one of these two sets, whether the following property holds:

$$f(X_0^+) \subseteq X_0^+ \cup X_0^-$$

I.e. whether this set either converges (shrinks with each timestep) or maps to its "sister" set, reflected across the axes. If the property also holds for the sister set, then the union of both sets is forward invariant. At each timestep, any point starting in either set will either remain in the set or map to the other set, and so on.


### Coming up with a forward invariant set (came up last meeting)

It is easy to imagine a relationship like the above that holds over any number of sets, i.e.

$$A \mapsto B \mapsto C \mapsto ... \mapsto A$$

Furthermore, any set in this sequence can be considered stable/forward-invariant, not just $A$. This gives us another way of thinking about the original verification problem. Rather than checking whether some starting set is stable by propagating it, *a network can be stable iff* 

$$\exists \Phi \mid A \subseteq \Phi \implies f(A) \subseteq \Phi$$

That is to say, $f$ can only be stable if there exists a finite set $\Phi$ that maps to itself under $f$. Therefore, rather than consider $A$ stable because it maps back to itself over some number of mappings, as presented earlier, the entire path between $A$ and itself is forward invariant. Any subset of that path is therefore equally stable, and any set not in $\Phi$ is unstable. In this framing of the stability problem, we ask whether there exists a $\Phi$ for which the statement holds. If we can find such a set, it may possible from there to determine whether that set is "reasonable" starting set for the system in question.

It may be possible to glean this sort of forward invariance information using SMT based verification (don't know how). Using reachability methods may be a way to do it also, and both will require domain knowledge to even begin. Not sure how this is actionable yet, if at all.

Worth noting that the set of all inputs, i.e., $\Phi = \mathbb{R}^d$ is forward invariant, so $\Phi$ has to at least be finite for this to be of any use...

## Image space

All of the above does not mention convolutions or image-space or anything like that. It would be perfectly alright to start by verifying properties on a network which takes in $[\theta, \dot\theta]$ directly, since this is a challenging enough problem. Particularly when we get to RNNs.

The image issue adds two layers of complexity. The first one is obvious, and that is the network size. instead of O(10) nodes we have O(10k), which even state of the art solvers have a hard time with. The other issue, which is perhaps more pernicious, is dealing with input/output sets. A set which is naturally convex/linear in state space is not in image space. For example, how can you represent the set of images corresponding to the angles $\theta \in [-\pi/36, \pi/36]$? Although it is a simple interval, you can't represent it in image-space in a compact or convex way. 

When verifying properties on images, most people resort to $L_1$ or $L_\infty$ norm properties, by turning pixels on and off or adding noise to them, but these don't have physical meaning like the kind we're looking for. I.e. they don't map to a range of states. That said, it would be fine to do something along these lines to start. E.g. for images within $\epsilon$, in an $L_\infty$ sense, of the image corresponding to $\theta = a$, show that the controller is stable. This kind of property corresponds roughly to a camera model with uniform noise. In general, you can specify whatever interval of values you want for each pixel so long as the interval contains the input image's value. We can also isolate particular pixels to allow noise in, and other not to. 

Another thing we can do, is consider some (probably small) arc, i.e. range of angles, and plot the entire range in image-space. Allowing each pixel in that arc to take on a value between 0 and 1 (1 being completely active), we define a set in image-space that definitely contains all images that come from the range of angles defining the arc. It also contains many other, irrelevant, images, including all possible combination of pixel activations for any collection of pixels within the arc. By setting certain other constraints on top of this one (such as an $L_1$ constraint requiring a minimum activation), we can maybe arrive at image space sets that are both actionable and physically relevant.
